<a href="https://colab.research.google.com/github/royadeh/Pytorch_samples/blob/main/ConveXt_description.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


ConvNext:
The authors start with Resnet50 and modernize it to make it like a vision transformer like Swin Transformer. 

Training technology: Besides the architecture of the network, the training methodology has a significant effect on the overall performance of the models.  Vision transformers use AdamW optimizer. So, Resnet was trained with similar training techniques as vision transformers are trained with. ( 76.1   78.8 )
 
Stage1 :Macro design
Stage ratio: it adjust the number of blocks in each stage. In ResNet we have 4 stages, Swin Transformer uses a ratio of 1:1:3:1 (so one block in the first stage, one in the second, third in the third one ...). Adjusting ResNet50 to this ratio ((3, 4, 6, 3) -> (3, 3, 9, 3)) results in a performance increase from 78.8% to 79.4%.
“Patchify” stem:
ResNet stem uses a very aggressive 7x7 conv and a maxpool to heavily downsample the input images. ransformers, on the other hand, employs a "patchify" stem, which means they embed the input images in patches. While the authors utilise a 4x4 patch built with a conv layer, Vision Transfomers uses extremely aggressive patching (16x16). The accuracy decreases from 79.4 to 79.5 percent, indicating that patching is effective.

Stege2: NesNetXt
NesNetXt uses depthwise grouped convolution for the 3x3 conv layer in the BottleNeck to reduce FLOPS. It reduces the accuracy but simultaneously widening the network improves performance substantially.

Stege3: Inverted bottleneck
This is the case for every Swin transformers in particular because in Swin transformer that work hierarchically, the output of 4 blocks get concatenated such that in the end, the hidden dimension increases 4 times. If you want to use this idea, we should use “inverted bottleneck” design with an expansion ratio of 4 used in convnet. It increases the performance too, if only by a little. 

Stage4: large kernel
A Cnn layer uses kernels to aggregates local info. In each layer, which is passed to the next layer that aggregates again local info., but now with a larger field of view. The receptive field becomes more global in each layer. However, transformers have a large field of view from the beginning. If the authors increase the resnet window size, then they should reach the same idea. It is clear that small kernel size hurt the performance. 

Stage5: Micro Design
 Replacing ReLU with GeLU (Gaussian error linear unit)
Since GELU is used by the most advanced transformers, why not use it in our model? The authors report the accuracy stays unchanged. In PyTorch GELU in nn.GELU.
Fewer activation functions
Our block has three activation functions. While, in Transformer block, there is only one activation function, the one inside the MLP block. The authors removed all the activations except for the one after the middle conv layer. This improves accuracy to 81.3% matching Swin-T!
Fewer normalization layers
Similar to activations, Transformers blocks have fewer normalization layers. The authors decide the remove all the BatchNorm and kept only the one before the middle conv.
Substituting BN with LN
Well, they substitute the BatchNorm layers with LinearyNorm. They note that doing so in the original ResNet hurts performance, but after all our changes, the performance increases to 81.5%
separate layers for downsampling.
ResNet uses the stride=2 conv to perform downsampling. Transformers have a distinct downsampling block, as do other conv nets. The authors added a downsampling block before the three convs using a 2x2 stride=2 conv and removed the stride=2. Prior to the downsampling procedure, normalisation is required to maintain stability throughout training. This module can be added to our ConvNexStage. Finally, we exceed Swin by 82.0 percent!

We use batch normalization layers to make values down between -1 and 1. 
 


